<a href="https://colab.research.google.com/github/sarkar-sayan/Feedback_Analysis-using-Machine_Learning_NLP_GenerativeAI/blob/main/POC_2_SFA_Threat_Check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install groq

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, FunctionTransformer
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import re
import nltk
import os
import pickle
from IPython.display import display

#from langdetect import detect
#from googletrans import Translator
# Initialize Google Translator
#translator = Translator()

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


In [ ]:
from groq import Groq
!export GROQ_API_KEY='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'

client = Groq(
    # This is the default and can be omitted
    api_key='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'
    #api_key=os.environ.get("GROQ_API_KEY"),
)

In [ ]:
# Text preprocessing function
# Translate text if not in English
#def translate_text_if_needed(text):
#    try:
#        language = detect(text)
#        if language != 'en':
#            translated = translator.translate(text, dest='en')
#            return translated.text
#        return text
#    except Exception as e:
#        return text

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    filtered_text = ' '.join([lemmatizer.lemmatize(word) for word in tokens if word not in stop_words])
    return filtered_text


In [ ]:
def prepare_data(dataset_url):
  df = pd.read_csv(dataset_url)
  # preprocess data accroding to survey form
  #df.drop('Timestamp = A12 + A4', axis=1, inplace=True)
  df = df.dropna(subset=["Timestamp = A12 + A4"])  # if Timestamp == null, that means empty row, i.e, the form has never been filled

  # Join all textual columns into one
  feedback_columns = [col for col in df.columns if 'Overall feedback' in col]
  df['All_feedback'] = df[feedback_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
  #df['All_feedback'] = df['All_feedback'].apply(translate_text_if_needed)
  df.drop(feedback_columns, axis=1, inplace=True)
  # Preprocess textual feedback
  df['All_feedback'] = df['All_feedback'].apply(preprocess_text)
  return df

In [ ]:
def train_test_model(data):
  X = data.drop('Final Label', axis=1)
  y = data['Final Label']

  # Train/Test Split
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
  # Define ColumnTransformer to separate numerical and text data processing
  preprocessor = ColumnTransformer(
      transformers=[
          ('num', StandardScaler(), X.select_dtypes(include='number').columns.tolist()),
          ('text', TfidfVectorizer(), 'All_feedback')
          ],
      remainder='drop'
      )

  # Define the pipeline
  pipeline = Pipeline([
      ('preprocessor', preprocessor),
      ('classifier', LogisticRegression(random_state=42))
      ])

  pipeline.fit(X_train, y_train)

  # Predict on the test set
  y_pred = pipeline.predict(X_test)

  #print("Accuracy:", accuracy_score(y_test, y_pred))
  #print("Classification Report:")
  #print(classification_report(y_test, y_pred))
  return pipeline

In [ ]:
def predict_feedback_proba(pipeline, sample):
    sample_df = pd.DataFrame([sample])
    sample_df = sample.to_frame().T
    y_prob = pipeline.predict_proba(sample_df)
    prob_pos = y_prob[0][1] * 100  # Probability of being positive
    prob_neg = y_prob[0][0] * 100  # Probability of being negative
    return prob_pos, prob_neg

In [ ]:
# Function to generate detailed feedback using OpenAI's GPT-4
def generate_detailed_feedback(numerical_data, textual_feedback, prob_pos, prob_neg):
    # Convert numerical_data to a string representation
    numerical_data_str = str(numerical_data)

    prompt = f"Using the following numerical ratings and feedback:\n\nNumerical Ratings: {numerical_data_str}\n\nTextual Feedback: {textual_feedback}\n\nProbability of being positive: {prob_pos:.2f}%\n\nProbability of being negative: {prob_neg:.2f}%\n\nPlease provide a very short final feedback according to the client, i.e, either positive or negative. Not both.Give output in the form of either Positive, Mostly Positive, Negative or Mostly Negative with the probability percentage. And then Improvements or Matters to focus upon:"
    chat_completion = client.chat.completions.create(
    messages = [
    {
        "role": "system",
        "content": "You are an intelligent and helpful assistant.",
    },
{
        "role": "user",
        "content": prompt, # Use the formatted prompt here
    },
    ],
    model="llama3-8b-8192" # Add the model keyword argument here
    )
    return chat_completion.choices[0].message.content.strip()  # Extract the content from the response


In [ ]:
def generate_feedback(pipeline, sample):
  # Example usage
  #prob_pos, prob_neg = predict_feedback_proba(pipeline, sample)
  #print(f"Sample:\n{sample}")
  #print(f"Probability of being positive: {prob_pos:.2f}%")
  #print(f"Probability of being negative: {prob_neg:.2f}%")

  numerical_data = sample.drop('All_feedback').to_dict()
  textual_feedback = sample['All_feedback']
  prob_pos, prob_neg = predict_feedback_proba(pipeline, sample)
  detailed_feedback = generate_detailed_feedback(numerical_data, textual_feedback, prob_pos, prob_neg)
  return detailed_feedback

In [ ]:
def get_model_ready():
  dataset_url = input("Enter the dataset URL: ")
  df = prepare_data(dataset_url)
  pipeline = train_test_model(df)
  return pipeline

In [ ]:
def get_feedback(pipeline, url):
  feedform_url = url
  data = prepare_data(feedform_url)
  df = pd.read_csv(feedform_url)
  df['Result'] = ""
  for i in range(len(data)):
    sample = data.iloc[i]
    print("\nFeedback: ", i+1)
    print("Client Name: ", data['Client Name'].iloc[i])
    pos, neg = predict_feedback_proba(pipeline, sample)
    df.at[i, 'Final Label'] = "positive" if pos > neg else "negative"
    feed = generate_feedback(pipeline, sample)
    df.at[i, 'Result'] = feed
    print(feed)
  #data = data.drop('All_feedback', axis=1)
  df.to_csv(feedform_url, index=False)

In [ ]:
def dump_load_classifier():
  pipeline = get_model_ready()
  # save the model to disk
  filename = '---/finalized_model.sav'
  pickle.dump(pipeline, open(filename, 'wb'))
  #load_pipeline = pickle.load(open(filename, 'rb'))
  return filename

def execute_one(url):
  filename = dump_load_classifier()     #if classifier not trained, uncomment
  #filename = input("Enter Pre-trained classifier url")    #if classifier already trained and saved, uncomment
  pipeline = pickle.load(open(filename, 'rb'))
  get_feedback(pipeline, url)

In [ ]:
def threat_check(df):
  unique_clients = df['Client Name'].unique()
  result_dict = {client: 0 for client in unique_clients}
  for index, row in df.iterrows():
    client_name = row['Client Name']
    final_label = row['Final Label']
    if final_label == 'negative':
      result_dict[client_name] += 1

  return [client_name for client_name, value in result_dict.items() if value > 2]

def filtered_threats_as_table(df, threat_dict):
    filtered_df = df[(df['Client Name'].isin(threat_dict)) & (df['Final Label'] == 'negative')]
    ordered_df = filtered_df.sort_values(by='Client Name')
    return ordered_df

def execute_two(df):
  threats = threat_check(df)
  threat_data = filtered_threats_as_table(df, threats)
  display(threat_data)

In [ ]:
# For displaying individual feedbacks without using Table
"""def display_threat_reviews(df, threat_dict):
    negative_reviews = {}
    for index, row in df.iterrows():
        client_name = row['Client Name']
        final_label = row['Final Label']
        result = row['Result']

        if client_name in threat_dict and final_label == 'negative':
            if client_name not in negative_reviews:
                negative_reviews[client_name] = [result]
            else:
                negative_reviews[client_name].append(result)
    # Print the negative reviews for each client
    for client_name, reviews in negative_reviews.items():
      print(f"Client: {client_name}")
      for review in reviews:
        print(f"  - {review}") """


In [ ]:
def main():
  feedform_url = input("Enter the URL where feedback data for generating analysis is stored: ")
  df = pd.read_csv(feedform_url)
  execute_one(feedform_url)           # for generating and placing review feeds into Output table
  execute_two(df)                     # for multiple negative reviews by same client

In [ ]:
if __name__ == "__main__":
  main()